<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/UV_Measurement_to_CSV_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Hier werden die Rohdaten des BTS2048-UV-WP in eine nutzbare CSV-Datei geschrieben**

Dokumentenname: UV_Measurement_to_CSV.ipynb




Es werden die OR0-Daten (NasaAmes-Format), die eigentlich für den Versand an das BFS gedacht sind umgewandelt und in eine CSV-Datei geschrieben. Aufgrund der großen Datenmenge geschieht dies für jeden Monat getrennt.

In [4]:
# Import der benötigten Module
import os, sys
import glob
import json
import datetime
import pandas as pd
import numpy as np
import csv
import re

from scipy.io import netcdf
from datetime import timedelta
from datetime import datetime

from google.colab import drive
from google.colab import files

Zu Beginn muss die Google-Drive eingerichtet werden, in der die Messdaten (OR0-Dateien) gespeichert sind. Danach werden alle verfügbaren Unterordner aufgerufen. So wird geprüft, ob der Mount richtig funktioniert hat.

In [5]:
drive.mount('/content/drive')

drive_path = '/content/drive/MyDrive'
# Durchsuche den Google Drive-Pfad
for root, dirs, files in os.walk(drive_path):
    for dir in dirs:
        # Gib den Namen des Unterordners aus
        print(os.path.join(root, dir))

Mounted at /content/drive
/content/drive/MyDrive/Colab_Notebooks
/content/drive/MyDrive/Colab Notebooks
/content/drive/MyDrive/Colab_Notebooks/CSV_Messdaten
/content/drive/MyDrive/Colab_Notebooks/CouchDB File
/content/drive/MyDrive/Colab_Notebooks/NasaAmes_Messdaten
/content/drive/MyDrive/Colab_Notebooks/NasaAmes_Messdaten/Data
/content/drive/MyDrive/Colab_Notebooks/NasaAmes_Messdaten/Data/22.07
/content/drive/MyDrive/Colab_Notebooks/NasaAmes_Messdaten/Data/22.08
/content/drive/MyDrive/Colab_Notebooks/NasaAmes_Messdaten/Data/22.09
/content/drive/MyDrive/Colab_Notebooks/NasaAmes_Messdaten/Data/22.10
/content/drive/MyDrive/Colab_Notebooks/NasaAmes_Messdaten/Data/22.11
/content/drive/MyDrive/Colab_Notebooks/NasaAmes_Messdaten/Data/22.12
/content/drive/MyDrive/Colab_Notebooks/NasaAmes_Messdaten/Data/23.01
/content/drive/MyDrive/Colab_Notebooks/NasaAmes_Messdaten/Data/23.02
/content/drive/MyDrive/Colab_Notebooks/NasaAmes_Messdaten/Data/23.03
/content/drive/MyDrive/Colab_Notebooks/NasaAmes_M

Danach werden die einzelnen OR0-Dateien (NasaAmes Format) geladen und umgewandelt.

Das NasaAmes Format: https://espoarchive.nasa.gov/content/Ames_Format_Specification_v20

Die Messdaten sind unter dem FFI (File Format Index) 2005 gespeichert. Dieser Standard ist durch die Nasa nicht dokumentiert. Deshalb nachfolgend ein eigener Parser, der die Daten in eine nutzbare CSV umwandelt.

**Beschreibung der einzelnen Dictionaries und deren Inhalt:**

- **file_names** = enthält die Dateinamen der einzelnen OR0-Dateien
- **file_content** = Enthält den Inhalt der OR0-Dateien
- **end_line_header** = enthält die Zeile an dem der Header endet

**Dateien einlesen:**

In [6]:
# Pfad zum Ordner mit den Dateien in Google Drive
folder_path = '/content/drive/MyDrive/Colab_Notebooks/NasaAmes_Messdaten/Data/22.06'

# OR0-Dateien im Ordner lesen
file_paths = glob.glob(folder_path + '/*.OR0')

# Liste für die Dateinamen erstellen
file_names = []

# Schleife über die Dateien
for file_path in file_paths:

    if os.path.getsize(file_path) > 100 * 1024:
      # Dateiname extrahieren
      file_name = os.path.splitext(os.path.basename(file_path))[0]

      # Datei öffnen und Inhalt lesen
      with open(file_path, 'r') as file:
          file_content = file.read()

      # Variable für die Datei erstellen
      globals()[file_name] = file_content

      file_names.append(file_name)

**Dataframe mit Wellenlängen erstellen:**
- Dieser wird später genutzt um die Spalten des Dataframe zu benennen.

In [7]:
def create_df_Wellenlaenge(start, end, step):
    # Erstelle eine Liste mit den gewünschten Werten
    numbers_list = [round(num, 3) for num in list(np.arange(start, end + step, step))]
    # Erstelle den Dataframe
    df = pd.DataFrame({'Wellenlaenge': numbers_list})

    return df

In [8]:
np_Wellenlaenge = np.round(np.arange(290.0, 420.05, 0.1), decimals = 1)
df_Wellenlaenge = pd.DataFrame({'Wellenlaenge': np_Wellenlaenge})

**String aufteilen in einzelne Zeile schreiben:**

- Zur weitern Verarbeitung müssen alle Elemente als einzelene Strings abgespeichet werden.

In [9]:
file_content = {}  # Dictionary für die Messungen erstellen

for file_name in file_names:
    file_variables = globals()[file_name]
    file_content[file_name] = file_variables

# Auf Variablen zugreifen und String in Zeilen aufteilen
for file_name, variable in file_content.items():
    file_content[file_name] = file_content[file_name].split('\n')

In [10]:
# die ersten 10 File Namen ausgeben und die ersten 20 Zeilen der ersten Messung
print(file_names[:10])
file_content['SA220615'][:20]

['SA220615', 'SA220616', 'SA220617', 'SA220618', 'SA220619', 'SA220620', 'SA220621', 'SA220622', 'SA220623', 'SA220624']


['NLHEAD=207 FFI=2005        ',
 'R.Janowitz',
 'DGUV',
 'ELONG=7.182 NLAT=50.780 HEIGHT=70m',
 's-UV-Mo-Net',
 '1 1',
 '2022 06 15  2022 06 15',
 '0 360',
 '1301',
 '1301',
 '  290.000  290.100  290.200  290.300  290.400  290.500  290.600  290.700',
 '  290.800  290.900  291.000  291.100  291.200  291.300  291.400  291.500',
 '  291.600  291.700  291.800  291.900  292.000  292.100  292.200  292.300',
 '  292.400  292.500  292.600  292.700  292.800  292.900  293.000  293.100',
 '  293.200  293.300  293.400  293.500  293.600  293.700  293.800  293.900',
 '  294.000  294.100  294.200  294.300  294.400  294.500  294.600  294.700',
 '  294.800  294.900  295.000  295.100  295.200  295.300  295.400  295.500',
 '  295.600  295.700  295.800  295.900  296.000  296.100  296.200  296.300',
 '  296.400  296.500  296.600  296.700  296.800  296.900  297.000  297.100',
 '  297.200  297.300  297.400  297.500  297.600  297.700  297.800  297.900']

**Header extrahieren:**

In [11]:
# Funktion um den Dateiheader zu extrahieren
def perform_action(file_variables, file_names):
    header_dict = {file_name: "" for file_name in file_names}
    end_line_header_fnc = 0

    for file_name, data in file_variables.items():
        for i, line in enumerate(data):
            header_dict[file_name] += line + "\n"
            if line.strip() == "Pyranometer: readout interval [secs]=5":
                end_line_header_fnc = i
                break

    return header_dict, end_line_header_fnc

In [12]:
file_header, end_line_header = perform_action(file_content, file_names)

**Nach Datum in Header suchen und in Datetime umwandeln:**

In [13]:
def extract_dates_from_dict(dictionary):
    dates = {}

    for key, value in dictionary.items():
        header_content = value  # Annahme: Der Wert im Dictionary enthält den Header-Inhalt
        header_split = header_content.split('\n')
        date_line = header_split[6].split()
        start_date = "-".join(date_line[:3])
        end_date = "-".join(date_line[3:])

        if start_date == end_date:
            date_object = datetime.strptime(start_date, "%Y-%m-%d")
            dates[key] = date_object
        else:
            dates[key] = "Error: Start and end dates are not the same"
            print("Error: Start and end dates are not the same")

    return dates

In [14]:
# dict welches das Datum jeder einzelnen Datei enthält:
date = extract_dates_from_dict(file_header)

In [15]:
date

{'SA220615': datetime.datetime(2022, 6, 15, 0, 0),
 'SA220616': datetime.datetime(2022, 6, 16, 0, 0),
 'SA220617': datetime.datetime(2022, 6, 17, 0, 0),
 'SA220618': datetime.datetime(2022, 6, 18, 0, 0),
 'SA220619': datetime.datetime(2022, 6, 19, 0, 0),
 'SA220620': datetime.datetime(2022, 6, 20, 0, 0),
 'SA220621': datetime.datetime(2022, 6, 21, 0, 0),
 'SA220622': datetime.datetime(2022, 6, 22, 0, 0),
 'SA220623': datetime.datetime(2022, 6, 23, 0, 0),
 'SA220624': datetime.datetime(2022, 6, 24, 0, 0),
 'SA220625': datetime.datetime(2022, 6, 25, 0, 0),
 'SA220626': datetime.datetime(2022, 6, 26, 0, 0),
 'SA220627': datetime.datetime(2022, 6, 27, 0, 0),
 'SA220628': datetime.datetime(2022, 6, 28, 0, 0),
 'SA220629': datetime.datetime(2022, 6, 29, 0, 0),
 'SA220630': datetime.datetime(2022, 6, 30, 0, 0)}

**Header aus Datensatz löschen:**

In [16]:
def remove_header(lines_content, end_line):
    lines_WO_header_fnc = {}
    lines_WO_header_fnc = lines_content.copy()

    for key, value in lines_WO_header_fnc.items():
        del value[:end_line+1]

    return lines_WO_header_fnc

In [17]:
lines_WO_header = remove_header(file_content, end_line_header)

In [18]:
print(lines_WO_header['SA220615'][:20])

['26460', '113 4.000E-02 9.999E+9', '2.000E+00 0.000E+00 999.9 9.999E+9 9.999E+9 ', '9.999E+9 9.999E+9 9.999E+9 9.999E+9', '9.999E+9 9.999E+9 9.999E+9 9.999E+9', '  0.0000E+00  0.0000E+00  0.0000E+00  0.0000E+00  0.0000E+00', '  0.0000E+00  0.0000E+00  0.0000E+00  0.0000E+00  0.0000E+00', '  0.0000E+00  0.0000E+00  0.0000E+00  0.0000E+00  0.0000E+00', '  0.0000E+00  0.0000E+00  0.0000E+00  0.0000E+00  0.0000E+00', '  0.0000E+00  0.0000E+00  0.0000E+00  0.0000E+00  0.0000E+00', '  0.0000E+00  0.0000E+00  0.0000E+00  0.0000E+00  0.0000E+00', '  0.0000E+00  0.0000E+00  0.0000E+00  0.0000E+00  0.0000E+00', '  0.0000E+00  0.0000E+00  0.0000E+00  0.0000E+00  0.0000E+00', '  0.0000E+00  0.0000E+00  0.0000E+00  0.0000E+00  0.0000E+00', '  0.0000E+00  0.0000E+00  0.0000E+00  0.0000E+00  0.0000E+00', '  0.0000E+00  0.0000E+00  0.0000E+00  0.0000E+00  0.0000E+00', '  0.0000E+00  0.0000E+00  0.0000E+00  0.0000E+00  0.0000E+00', '  0.0000E+00  0.0000E+00  0.0000E+00  0.0000E+00  0.0000E+00', '  0.0

**Elemente aufteilen:**

In [19]:
def flatten_and_split(input_list):
    result = []
    for sublist in input_list:
        elements = sublist.split()
        result.extend(elements)
    return result

In [20]:
def process_dict(input_dict):
    processed_dict = {}
    for key, value in input_dict.items():
        processed_value = flatten_and_split(value)
        sublists = []
        sublist = []
        for element in processed_value:
            if element.isdigit() and len(element) == 5:
                if sublist:
                    sublists.append(sublist)
                    sublist = []
            sublist.append(element)
        if sublist:
            sublists.append(sublist)
        processed_dict[key] = sublists
    return processed_dict

In [21]:
processed_dict = process_dict(lines_WO_header)

In [22]:
print(processed_dict['SA220615'][:2])

[['26460', '113', '4.000E-02', '9.999E+9', '2.000E+00', '0.000E+00', '999.9', '9.999E+9', '9.999E+9', '9.999E+9', '9.999E+9', '9.999E+9', '9.999E+9', '9.999E+9', '9.999E+9', '9.999E+9', '9.999E+9', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0000E+00', '0.0

**Zeitstempel in Datensatz finden:**

In [23]:
def find_5_digit_integers(input_list):
    result = []
    for sublist in input_list:
        for element in sublist:
            if isinstance(element, str) and element.isdigit() and len(element) == 5:
                result.append(element)
    return result

In [24]:
def find_5_digit_integers_in_dict(input_dict):
    result_dict = {}
    for key, value in input_dict.items():
        result_dict[key] = find_5_digit_integers(value)
    return result_dict

In [25]:
result_dict = find_5_digit_integers_in_dict(processed_dict)

In [26]:
result_dict

{'SA220615': ['26460',
  '26580',
  '26700',
  '26820',
  '26940',
  '27060',
  '27180',
  '27300',
  '27420',
  '27540',
  '27660',
  '27780',
  '27900',
  '29700',
  '29820',
  '29940',
  '30060',
  '30180',
  '30300',
  '30420',
  '30540',
  '30660',
  '30780',
  '30900',
  '31020',
  '31140',
  '31260',
  '31380',
  '31500',
  '31620',
  '31740',
  '31860',
  '31980',
  '32100',
  '32220',
  '32340',
  '32460',
  '32580',
  '32700',
  '32820',
  '32940',
  '33060',
  '33180',
  '33300',
  '33420',
  '33540',
  '33660',
  '33780',
  '33900',
  '34020',
  '34140',
  '34260',
  '34380',
  '34500',
  '34620',
  '34740',
  '34860',
  '34980',
  '35100',
  '35220',
  '35340',
  '35460',
  '35580',
  '35700',
  '35820',
  '35940',
  '36060',
  '36180',
  '36300',
  '36420',
  '36540',
  '36660',
  '36780',
  '36900',
  '37020',
  '37140',
  '37260',
  '37380',
  '37500',
  '37620',
  '37740',
  '37860',
  '37980',
  '38100',
  '38340',
  '38460',
  '38580',
  '38700',
  '38820',
  '38940'

**Veraltet: Zeitstempel in eine Uhrzeit umwandeln:**

In [27]:
def seconds_to_time(seconds):
    hours = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
    return f"{hours:02}:{minutes:02}:{seconds:02}"

In [28]:
def convert_and_store_first_entries(input_dict):
    result_datetime_dict = {}
    for key, value in input_dict.items():
        date = datetime.strptime(key[2:],"%y%m%d")# Extrahiere das Datum aus dem Schlüssel
        times = []
        for sublist in value:
            if sublist:  # Überprüfe, ob die Liste nicht leer ist
                first_entry_seconds = int(sublist[0])  # Der erste Eintrag in der Liste
                time = seconds_to_time(first_entry_seconds)
                times.append(time)
            else:
                times.append(None)
        result_datetime_dict[date] = times
    return result_datetime_dict

In [29]:
result_first_entries_dict = convert_and_store_first_entries(processed_dict)

In [30]:
print(result_first_entries_dict)

{datetime.datetime(2022, 6, 15, 0, 0): ['07:21:00', '07:23:00', '07:25:00', '07:27:00', '07:29:00', '07:31:00', '07:33:00', '07:35:00', '07:37:00', '07:39:00', '07:41:00', '07:43:00', '07:45:00', '08:15:00', '08:17:00', '08:19:00', '08:21:00', '08:23:00', '08:25:00', '08:27:00', '08:29:00', '08:31:00', '08:33:00', '08:35:00', '08:37:00', '08:39:00', '08:41:00', '08:43:00', '08:45:00', '08:47:00', '08:49:00', '08:51:00', '08:53:00', '08:55:00', '08:57:00', '08:59:00', '09:01:00', '09:03:00', '09:05:00', '09:07:00', '09:09:00', '09:11:00', '09:13:00', '09:15:00', '09:17:00', '09:19:00', '09:21:00', '09:23:00', '09:25:00', '09:27:00', '09:29:00', '09:31:00', '09:33:00', '09:35:00', '09:37:00', '09:39:00', '09:41:00', '09:43:00', '09:45:00', '09:47:00', '09:49:00', '09:51:00', '09:53:00', '09:55:00', '09:57:00', '09:59:00', '10:01:00', '10:03:00', '10:05:00', '10:07:00', '10:09:00', '10:11:00', '10:13:00', '10:15:00', '10:17:00', '10:19:00', '10:21:00', '10:23:00', '10:25:00', '10:27:00', 

**dict in Dataframe speichern:**

In [31]:
# Erstelle eine leere Liste, um die Zeilen für den DataFrame aufzunehmen
data_rows = []
df_Messdaten = []


# Iteriere durch das verschachtelte Dictionary und erstelle Zeilen für den DataFrame
for key, value in processed_dict.items():
    for sublist in value:
        data_rows.append([key] + sublist)

# Definiere Spaltennamen für den DataFrame
columns = ['Datum'] + [f'Wert{i}' for i in range(1, len(data_rows[0]))]

# Erstelle den Pandas DataFrame
df_Messdaten = pd.DataFrame(data_rows, columns=columns)

In [32]:
print(len(df_Wellenlaenge), len(df_Messdaten.columns))

1301 1319


In [33]:
# Bestimme die Indexposition, ab der die neuen Spaltennamen zugewiesen werden sollen
start_index = 18  # Beispiel: Beginne ab der dritten Spalte

# Extrahiere die neuen Spaltennamen aus dem zweiten DataFrame
new_column_names = df_Wellenlaenge['Wellenlaenge'].tolist()

# Ändere die Spaltennamen des DataFrames ab der angegebenen Indexposition
for i, new_name in enumerate(new_column_names):
    df_Messdaten.columns.values[start_index + i] = new_name

In [34]:
df_Messdaten

,Datum,Wert1,Wert2,Wert3,Wert4,Wert5,Wert6,Wert7,Wert8,Wert9,...,419.1,419.2,419.3,419.4,419.5,419.6,419.7,419.8,419.9,420.0
0,SA220615,26460,113,4.000E-02,9.999E+9,2.000E+00,0.000E+00,999.9,9.999E+9,9.999E+9,...,6.9350E-01,6.9191E-01,6.8556E-01,6.6990E-01,6.5661E-01,6.4664E-01,6.4017E-01,6.3688E-01,6.3653E-01,6.4241E-01
1,SA220615,26580,111,4.000E-02,9.999E+9,2.000E+00,0.000E+00,999.9,9.999E+9,9.999E+9,...,7.2334E-01,7.2015E-01,7.1598E-01,6.9777E-01,6.9125E-01,6.7876E-01,6.6992E-01,6.6380E-01,6.6182E-01,6.6927E-01
2,SA220615,26700,111,4.000E-02,9.999E+9,2.000E+00,0.000E+00,999.9,9.999E+9,9.999E+9,...,7.2460E-01,7.1990E-01,7.0922E-01,6.9384E-01,6.8664E-01,6.7670E-01,6.6620E-01,6.6193E-01,6.6160E-01,6.7178E-01
3,SA220615,26820,99,4.000E-02,9.999E+9,2.000E+00,0.000E+00,999.9,9.999E+9,9.999E+9,...,7.3936E-01,7.3744E-01,7.2915E-01,7.0938E-01,7.0058E-01,6.9258E-01,6.8047E-01,6.7285E-01,6.7156E-01,6.7532E-01
4,SA220615,26940,99,4.000E-02,9.999E+9,2.000E+00,0.000E+00,999.9,9.999E+9,9.999E+9,...,7.4128E-01,7.4184E-01,7.3163E-01,7.1063E-01,7.0125E-01,6.9214E-01,6.8244E-01,6.7641E-01,6.7705E-01,6.8511E-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7303,SA220630,67740,95,4.000E-02,9.999E+9,2.000E+00,0.000E+00,999.9,9.999E+9,9.999E+9,...,1.7621E-02,1.7573E-02,1.7332E-02,1.6969E-02,1.6789E-02,1.6625E-02,1.6391E-02,1.6304E-02,1.6265E-02,1.6429E-02
7304,SA220630,67860,95,4.000E-02,9.999E+9,2.000E+00,0.000E+00,999.9,9.999E+9,9.999E+9,...,1.5729E-02,1.5730E-02,1.5471E-02,1.5085E-02,1.4909E-02,1.4752E-02,1.4533E-02,1.4590E-02,1.4665E-02,1.4806E-02
7305,SA220630,67980,103,4.000E-02,9.999E+9,2.000E+00,0.000E+00,999.9,9.999E+9,9.999E+9,...,1.5218E-02,1.5181E-02,1.4969E-02,1.4715E-02,1.4497E-02,1.4267E-02,1.4077E-02,1.3996E-02,1.4004E-02,1.4091E-02
7306,SA220630,68100,103,4.000E-02,9.999E+9,2.000E+00,0.000E+00,999.9,9.999E+9,9.999E+9,...,1.7840E-02,1.7820E-02,1.7549E-02,1.7234E-02,1.6933E-02,1.6672E-02,1.6487E-02,1.6417E-02,1.6472E-02,1.6619E-02


In [35]:
# Funktion zum Konvertieren des Datumsformats
def convert_date_format(date_str):
    date_str = date_str[2:]  # Entferne das "SA"-Präfix
    date_obj = datetime.strptime(date_str, "%y%m%d")
    return date_obj.strftime("%y%m%d")

# Wende die Funktion auf die Spalte an
df_Messdaten['Datum'] = df_Messdaten['Datum'].apply(convert_date_format)

print(df_Messdaten)

       Datum  Wert1 Wert2      Wert3     Wert4      Wert5      Wert6  Wert7  \
0     220615  26460   113  4.000E-02  9.999E+9  2.000E+00  0.000E+00  999.9   
1     220615  26580   111  4.000E-02  9.999E+9  2.000E+00  0.000E+00  999.9   
2     220615  26700   111  4.000E-02  9.999E+9  2.000E+00  0.000E+00  999.9   
3     220615  26820    99  4.000E-02  9.999E+9  2.000E+00  0.000E+00  999.9   
4     220615  26940    99  4.000E-02  9.999E+9  2.000E+00  0.000E+00  999.9   
...      ...    ...   ...        ...       ...        ...        ...    ...   
7303  220630  67740    95  4.000E-02  9.999E+9  2.000E+00  0.000E+00  999.9   
7304  220630  67860    95  4.000E-02  9.999E+9  2.000E+00  0.000E+00  999.9   
7305  220630  67980   103  4.000E-02  9.999E+9  2.000E+00  0.000E+00  999.9   
7306  220630  68100   103  4.000E-02  9.999E+9  2.000E+00  0.000E+00  999.9   
7307  220630  68220    95  4.000E-02  9.999E+9  2.000E+00  0.000E+00  999.9   

         Wert8     Wert9  ...       419.1       419

In [36]:

def save_dataframe_to_drive(dataframe, folder_path, filename):

    # Erstelle den vollen Pfad zur Datei
    full_path = os.path.join(folder_path, filename)

    # Speichere den DataFrame als CSV-Datei auf Google Drive
    dataframe.to_csv(full_path, index=False)

    print(f'Der DataFrame wurde als {filename} in {folder_path} auf Google Drive gespeichert.')

In [37]:
save_dataframe_to_drive(df, '/content/drive/My Drive/', 'output.csv')

NameError: ignored